# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [28]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_GPT_2="gpt-5"
MODEL_LLAMA = 'llama3.2'

In [6]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key found and looks good so far!


In [7]:
system_prompt = "You are a technical expert who specializes in providing detailed and accurate information about various programming technologies and their applications. Your task is to assist users in understanding complex technical concepts or lines of code and to provide guidance on best practices. Your tone is very informational, helpful. You also use light humor to keep the mood light. Always answer in Markdown format."


In [8]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [23]:
# Get gpt-4o-mini to answer, with streaming
def format_prompt(prompt, model=MODEL):
    return {
        "model": model,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        "stream": False
    }


In [31]:
def ask_gpt(prompt):
    formatted_prompt = format_prompt(prompt, model=MODEL)
    response = openai.chat.completions.create(**formatted_prompt)

    content = response.choices[0].message.content
    return Markdown(content)


In [32]:
display(ask_gpt(question))

Certainly! Let's break down the code you provided:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

### Explanation:

1. **Context of Usage**:
   - This code appears to be part of a generator function, given the use of the `yield` keyword. A generator function is a special type of function that allows you to iterate over a sequence of values without storing the entire sequence in memory.

2. **Comprehension**:
   - Inside the `yield from` statement, we have a set comprehension:
     - `{...}` denotes that we are creating a set (a collection of unique items).
     - The expression `book.get("author")` attempts to obtain the value associated with the key `"author"` from each `book` dictionary in the `books` iterable.
     - The `for book in books` iterates over each item in the `books` collection.
     - The `if book.get("author")` condition filters out any `book` dictionaries that do not have an `"author"` key or have a `None` value for the `"author"` key.

3. **Purpose**:
   - The purpose of this line is to yield unique author names from the `books`. 
   - When you use `yield from` followed by an iterable, the generator will yield each item from that iterable one-by-one, which means each unique author will be produced when iterating over the generator.

### Why Make These Choices:

- **Using a Set**: By using a set comprehension, the code automatically ensures that only unique authors are collected. If multiple books have the same author, the set will include that author only once.
  
- **Handling Missing Authors**: The conditional `if book.get("author")` allows the function to gracefully skip any `book` dictionaries that are missing the `"author"` key or have a falsy value (like an empty string).

### Example:
Suppose `books` is defined as follows:

```python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": "Author A"},
    {"title": "Book 4"},  # No author
    {"title": "Book 5", "author": None}  # No author
]
```

Using the provided line would yield:
- **Output**: `Author A`, `Author B`

### Summary:
In summary, this line of code effectively gathers all unique authors from a list of book dictionaries while skipping those entries that don't have a valid author. It’s efficient and elegant—just like a well-polished novel! 📚✨ 

If you have further questions or need more clarity, feel free to ask!

In [34]:
def stream_ask_gpt(prompt):
    formatted_prompt = format_prompt(prompt)
    formatted_prompt["stream"] = True
    stream = openai.chat.completions.create(**formatted_prompt)

    completion = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        if chunk:
            completion += chunk.choices[0].delta.content or ''
            #completion = completion.replace("```","").replace("markdown", "")
            display_handle.update(Markdown(completion))


In [35]:
stream_ask_gpt(question)

Certainly! Let's break down the code snippet you provided, which uses both a generator and a set comprehension.

### Code Explanation

```python
yield from {book.get("author") for book in books if book.get("author")}
```

1. **`yield from`**:
   - This is a statement used to delegate part of a generator's operations to another iterable. Instead of using a loop to yield values one by one, `yield from` allows for more concise code and automatically handles closing and cleaning up the generator/function called.
   
2. **Set Comprehension**:
   - The expression `{book.get("author") for book in books if book.get("author")}` is a set comprehension. This creates a new set that contains unique authors from the `books` iterable.
   - The `for book in books` part iterates through each `book` object in the `books` collection (which is expected to be a list or some other iterable of dictionaries).
   - Inside the comprehension:
     - `book.get("author")`: Retrieves the value associated with the `author` key for each `book`. If the key does not exist, `None` is returned.
     - The `if book.get("author")` condition filters out entries where the `author` key does not exist or has a value of `None`, ensuring that only valid authors are included in the set.

### Purpose of the Code

- **What it does**: This line of code collects all unique authors from the `books` list and yields them one by one. The use of a set removes any duplicate author names, meaning you'll only get one instance of each author, even if they authored multiple books.
  
- **Why**: It's a concise way to generate a sequence of unique authors from a potentially large collection of books, while also ensuring that you're not including any empty or invalid author entries. This can be particularly useful in contexts such as filtering data for reporting, displaying author names in a UI, or any situation where you need a distinct list of authors.

### Example

Imagine you have the following list of books:

```python
books = [
    {"title": "Book 1", "author": "Alice"},
    {"title": "Book 2", "author": "Bob"},
    {"title": "Book 3", "author": "Alice"},
    {"title": "Book 4"},  # No author
    {"title": "Book 5", "author": None}  # Author is None
]
```

When you run the code, it will yield:

- `Alice`
- `Bob`

No duplicates and no empty authors!

### A Light-hearted Note

Just remember, this snippet is all about getting the "author-ity" in your book collection—so don't let any empty entries bookend your list! 📚😄

In [24]:
# Get Llama 3.2 to answer
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

def ask_llama(prompt):
    response = ollama_via_openai.chat.completions.create(**format_prompt(prompt, "llama3.2"))
    return Markdown(response.choices[0].message.content)

In [25]:
display(ask_llama(question))

**Breaking Down the Mystery Code**
=====================================

Let's dive into the world of generators and iterators with a snippet of Python code:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

### What is it doing?

This line of code uses a few advanced concepts to filter and extract data from a list of dictionaries. Let's break it down step by step:

*   `{...}`: This is a dictionary comprehension, which creates an anonymous dictionary.
*   `for book in books`: We're iterating over the `books` collection with each item being a dictionary (`book`).
*   `if book.get("author")`: We're filtering out dictionaries that don't have an "author" key. The `.get()` method returns undefined for cases where the given key doesn't exist, so we use this to skip them.
*   `yield from {...}`: This is where things get interesting! The `yield from` syntax allows us to delegate to another iterable (in this case, another dictionary comprehension) and have it yielded.

### What does `yield from` do?

When Python sees the `yield from` expression, it will " Delegate" the iteration of the sub-iteration to the `yield *` object. The yields produced by the inner generator are in turn passed on as the next Yield's yield arguments

In summary, this line of code is generating a dictionary containing only the values related to authors (`"author"` key) from each book dictionary, while skipping any items without an "author".

### Example Use Case:

Suppose you have a `books` list with dictionaries representing books:
```python
books = [
    {"id": 1, "title": "The Great", "author": "Jane"},
    {"id": 2, "title": "Of Mice and Men", "author": "John"},
    {"id": 3},  # No author in this one!
    {"id": 4, "title": "1984", "author": "George Orwell"}
]
```

```python
# Usage example:
def process_books(books):
	for book_author in yield from {book.get("author") for book in books if book.get("author")}:
		print(f"{book.get('id')}: {book.get('title')} (written by: {book_author})")

# Initialize the dictionary
process_books(books)
```

In this simplified case, `process_books` will simply iterate over each book's `author`, printing them out.


This code appears in the Python programming language version 3.5 and above